# Software Analysis - 02 - Execution Time

''Time is money!'' - that is especially true for applications where performance is a valuable property, e.g. if costs of cloud computation resources matter or if a software should run on the smallest/cheapest possible hardware platform. In these cases, it might be necessary to optimize the execution time of an application.

In the following, the analysis and optimization of an application's execution time is explained.

Shell command calls are handled by the Python module ``subprocess`` (https://docs.python.org/3/library/subprocess.html).

And shell output can be filtered by regular expressions with the Python module ``re`` (https://docs.python.org/3/library/re.html).

In [ ]:
import subprocess
import re

## Measuring Execution Time with ``time``

The Linux command line tool ``time``, specifically ``/usr/bin/time``, can be used to get a simple time measurement for the whole execution of a binary. The option ``-v`` outputs all available information. In the following examples, ``sleep 1`` is used as a test binary that waits for 1 second without much processing.

In [ ]:
time = subprocess.run(['/usr/bin/time', '-v', 'sleep', '1'], cwd='./', capture_output=True, text=True)
print(time.stderr)

The output of ``time`` can also be formatted by the ``--format`` option. For details, see https://linux.die.net/man/1/time.

In [ ]:
time = subprocess.run(['/usr/bin/time', '--format=REAL: %e  KERNEL: %S  USER: %U', 'sleep', '1'], cwd='./', capture_output=True, text=True)
print(time.stderr)

## Measuring Execution Time with ``perf``

The tool ``perf`` (https://perf.wiki.kernel.org) is a powerful choice for binary execution time analysis.

To enable ``perf`` one has to enable access to performance counter events in Linux (with ``root`` privileges):

``echo 0 > /proc/sys/kernel/perf_event_paranoid``

Of course, this can also be done outside of this Jupyter notebook in your shell!

In [ ]:
import os

sudo_password = 'rustlearner'
echo_command = 'sudo -S sh -c \'echo 0 > /proc/sys/kernel/perf_event_paranoid\''
os.system('echo ' + sudo_password + ' | ' + echo_command)

cat = subprocess.run(['cat', '/proc/sys/kernel/perf_event_paranoid'], cwd='./', capture_output=True, text=True)
print('perf_event_paranoid =', cat.stdout)

Basic information about the execution of a program can be obtained by calling it with ``perf stat``.

The following example measures the command ``sleep 1``.

Note that some performance counter values might be ``<not supported>`` within virtual machines or special processor platforms.

In [ ]:
perf = subprocess.run(['perf', 'stat', 'sleep', '1'], cwd='./', capture_output=True, text=True)
print(perf.stderr)

Rgular expressions might be useful to filter the output of ``perf stat``.

In [ ]:
perf = subprocess.run(['perf', 'stat', 'sleep', '1'], cwd='./', capture_output=True, text=True)
print('ELAPSED SECONDS:', re.search('([0-9]*,[0-9]*).seconds.time', perf.stderr).group(1))

## Distribution of Time Measurements

Keep in mind that every time measurement will have some noise involved, e.g. originating from operating system processes, I/O behavior, or hardware peculiarities.

If one measures the same tool several times, one might get several slightly different results.

In [ ]:
for i in range(5):
    perf = subprocess.run(['perf', 'stat', 'sleep', '1'], cwd='./', capture_output=True, text=True)
    print('TRY', i, ':', re.search('([0-9]*,[0-9]*).seconds.time', perf.stderr).group(1))

Therefore, it is reasonable to perform several time measurements and present the results in a distribution plot.

The plotting of distributions can be achieved with ``matplotlib``, while the module ``tqdm`` is able to provide a progress bar during the measurement process.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from tqdm import tqdm

In the following, the (useless) operation ``dd if=/dev/urandom of=/dev/null bs=16M count=1`` is measured, that moves 16 megabytes of random data into ``/dev/null``.

In [ ]:
n_measurements = 8*8
time_data = []
for i in tqdm(range(n_measurements)):
    perf = subprocess.run(['perf', 'stat', 'dd', 'if=/dev/urandom', 'of=/dev/null', 'bs=16M', 'count=1'], cwd='./', capture_output=True, text=True)
    time_str = re.sub(',', '.', re.search('([0-9]*,[0-9]*).seconds.time', perf.stderr).group(1))
    time_data.append(float(time_str))

Let's print the collected time data, the average, the min and max values and the distribution of all values.

In [ ]:
print('ALL MEASURED EXECUTION TIMES:')
for i in range(n_measurements//8):
    for j in range(8):
        print('{:03.5f}'.format(time_data[i*8+j]), end = ' ')
    print('')
print('')

print('AVERAGE EXECUTION TIME:', end=' ')
print('{:03.6f}'.format((sum(time_data)/len(time_data))))

print('MINIMUM EXECUTION TIME:', end=' ')
print('{:03.6f}'.format(min(time_data)))

print('MAXIMUM EXECUTION TIME:', end=' ')
print('{:03.6f}'.format(max(time_data)))

plt.hist(time_data, bins=32)
plt.xlabel('time / s')
plt.ylabel('frequency of occurence')
plt.show()

## Analysis of Prime Checking Applications in Rust

Checking numbers for primality is a common algorithm implementation case that leaves a lot of space for optimizations.

This exercise uses different versions of a command line tool for prime checking.

In [ ]:
prime_check_tools = [
    'prime_check_lookup_table',
    'prime_check_dyngen_table',
    'prime_check_no_table',
]

Let's compile all versions.

In [ ]:
print('### Compiling Prime Check Tools ###')
for tool in prime_check_tools:
  cargo = subprocess.run(['cargo', 'build', '--release'], cwd='./' + tool + '/')
  print('Compiling', '{:<24}'.format(tool), '--> Return Code:', cargo.returncode)

### Execution Time Analysis with 1 Request

In the following, all implementations are tested for their execution time when issuing one single prime checking request.

In [ ]:
numbers_to_check = ['1', '2', '11', '111', '1111', '11111', '111111', '999983']
time_data = [[] for tool in prime_check_tools]

for t in range(len(prime_check_tools)):
    tool = prime_check_tools[t]
    for n in tqdm(range(len(numbers_to_check))):
        perf = subprocess.run(['perf', 'stat', './' + tool, numbers_to_check[n]],
                              cwd='./' + tool + '/target/release/', capture_output=True, text=True)
        time_str = re.sub(',', '.', re.search('([0-9]*,[0-9]*).seconds.time', perf.stderr).group(1))
        time_data[t].append(float(time_str))

Let's plot the distributions and print the relevant data.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
ax_colors = ['brown', 'goldenrod', 'teal']

for t in range(len(prime_check_tools)):
    tool = prime_check_tools[t]
    ax = axs[t]
    
    # plot distribution
    ax.set_title(tool)
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.2e'))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax.xaxis.set_major_locator(plt.MaxNLocator(4))
    ax.yaxis.set_major_locator(plt.MaxNLocator(5))
    ax.hist(time_data[t], bins=16, color=ax_colors[t])
    ax.set_xlabel('time / s')
    ax.set_ylabel('frequency of occurence')
    
    # print data
    print('###', tool, '###')
    print('AVG:', '{:03.6f}'.format((sum(time_data[t])/len(time_data[t]))), end=' ')
    print('MIN:', '{:03.6f}'.format(min(time_data[t])), end=' ')
    print('MAX:', '{:03.6f}'.format(max(time_data[t])))
    
plt.show()

### Execution Time Analysis with a High Number of Requests

It is interesting to analyze, if the provided tools behave differently regarding performance and efficiency when fed with a high number of numbers to check for primality.

For such testing purposes, the module ``random`` (https://docs.python.org/3/library/random.html) can be useful to generate random test data.

In [ ]:
import random

Since the provided tools take numbers as command line arguments, one has to care about the maximum size of the shell string. On many Linux systems, it is only possible to pass 128 x 1024 = 131072 characters to the shell. Assuming 6 characters per number plus one space, one can probably check only about 131072 / 7 = 18724 numbers with a single call.

Therefore, in the following, all implementations are tested for their execution time when confronted with a near-maximum number of requests (18700).

The 18700 numbers to check are randomly choosen from the range between 0 and 999999.

In [ ]:
n_measurements = 8
n_numbers = 18700
numbers_to_check = [random.sample(range(1000000), n_numbers) for n in range(n_measurements)]
time_data = [[] for tool in prime_check_tools]

for t in range(len(prime_check_tools)):
    tool = prime_check_tools[t]
    for n in tqdm(range(n_measurements)):
        perf = subprocess.run(['perf', 'stat', './' + tool] + [str(x) for x in numbers_to_check[n]],
                              cwd='./' + tool + '/target/release/', capture_output=True, text=True)
        time_str = re.sub(',', '.', re.search('([0-9]*,[0-9]*).seconds.time', perf.stderr).group(1))
        time_data[t].append(float(time_str))

The following code generates the plots and relevant numbers.

In [ ]:
fig, axs = plt.subplots(1, len(prime_check_tools), figsize=(15, 5))
ax_colors = ['brown', 'goldenrod', 'teal']

for t in range(len(prime_check_tools)):
    tool = prime_check_tools[t]
    ax = axs[t]
    
    # plot distribution
    ax.set_title(tool)
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.2e'))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax.xaxis.set_major_locator(plt.MaxNLocator(4))
    ax.yaxis.set_major_locator(plt.MaxNLocator(5))
    ax.hist(time_data[t], bins=16, color=ax_colors[t])
    ax.set_xlabel('time / s')
    ax.set_ylabel('frequency of occurence')
    
    # print data
    print('###', tool, '###')
    print('AVG:', '{:03.6f}'.format((sum(time_data[t])/len(time_data[t]))), end=' ')
    print('MIN:', '{:03.6f}'.format(min(time_data[t])), end=' ')
    print('MAX:', '{:03.6f}'.format(max(time_data[t])))
    
plt.show()